## DIA 016: Fine-Turning Avanzado del Modelo

El Fine-Tuning avanzado implica descongelar algunas de las capas superiores del modelo base (VGG16) para permitir que el modelo aprenda características más específicas del nuevo conjunto de datos (MNIST). Esto puede mejorar la precisión y la capacidad de generalización del modelo.

In [ ]:
# 1. Importación de Librerías
# ------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:
# 2. Cargar el Mejor Modelo Guardado
# ------------------------------------

# Cargar el mejor modelo guardado durante el entrenamiento anterior
mejor_modelo = tf.keras.models.load_model('mejor_modelo.h5')
print("Mejor modelo cargado exitosamente.")


In [ ]:
# 3. Descongelar las Capas Superiores de VGG16 para Fine-Tuning
# ------------------------------------

# Obtener la capa base del modelo (VGG16)
base_model = mejor_modelo.layers[0]

# Mostrar el nombre de las capas para identificar las capas a descongelar
for i, layer in enumerate(base_model.layers):
    print(f"{i}: {layer.name} - {'Entrenable' if layer.trainable else 'Congelada'}")

# Decidir cuántas capas descongelar.
# Por ejemplo, descongelar las últimas 4 capas del bloque5
for layer in base_model.layers:
    if layer.name.startswith('block5'):
        layer.trainable = True
    else:
        layer.trainable = False

# Verificar qué capas están entrenables
print("\nEstado de las capas del modelo base después de descongelar:")
for layer in base_model.layers:
    print(f"{layer.name}: {'Entrenable' if layer.trainable else 'Congelada'}")


In [ ]:
# 4. Recompilar el Modelo con una Tasa de Aprendizaje Reducida
# ------------------------------------

# Recompilar el modelo para que los cambios en la entrenabilidad de las capas tengan efecto
# Usar una tasa de aprendizaje más baja para el fine-tuning
mejor_modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])


In [ ]:
# 5. Reentrenar el Modelo con Fine-Tuning
# ------------------------------------

# Definir callbacks adicionales
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=1e-7, verbose=1)

# Reentrenar el modelo con fine-tuning
history_ft = mejor_modelo.fit(
    train_ds,
    epochs=10,  # Puedes ajustar el número de épocas según sea necesario
    validation_data=val_ds,
    callbacks=[early_stop, checkpoint, reduce_lr]
)


In [ ]:
# 6. Evaluar y Visualizar el Rendimiento Mejorado
# ------------------------------------

# Evaluar el modelo después del fine-tuning
test_loss_ft, test_acc_ft = mejor_modelo.evaluate(val_ds, verbose=2)
print(f'\nPrecisión en el conjunto de prueba después del fine-tuning: {test_acc_ft:.4f}')

# Graficar precisión y pérdida durante el fine-tuning
plt.figure(figsize=(12, 4))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(history_ft.history['accuracy'], label='Entrenamiento')
plt.plot(history_ft.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el Fine-Tuning')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

# Pérdida
plt.subplot(1, 2, 2)
plt.plot(history_ft.history['loss'], label='Entrenamiento')
plt.plot(history_ft.history['val_loss'], label='Validación')
plt.title('Pérdida durante el Fine-Tuning')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.show()

In [ ]:
# 7. Guardar el Modelo Final
# ------------------------------------

# Guardar el modelo completamente entrenado
mejor_modelo.save('modelo_cnn_transfer_learning_mnist_final.h5')
print("Modelo final guardado como 'modelo_cnn_transfer_learning_mnist_final.h5'")


In [ ]:
# 8. Hacer Predicciones con el Modelo Final
# ------------------------------------

# Hacer predicciones sobre el conjunto de prueba
predicciones_ft = mejor_modelo.predict(val_ds)
clases_predichas_ft = np.argmax(predicciones_ft, axis=1)

# Mostrar predicción para la primera imagen del conjunto de prueba
indice = 0
print(f"Etiqueta real: {y_test[indice]}")
print(f"Predicción: {clases_predichas_ft[indice]}")
print(f"Probabilidades: {predicciones_ft[indice]}")

# Visualizar la imagen con su predicción
def mostrar_prediccion(matriz, etiquetas, predicciones, indice):
    plt.figure(figsize=(2,2))
    plt.imshow(matriz[indice].reshape(28, 28), cmap='gray')
    plt.title(f"Real: {etiquetas[indice]}\nPred: {predicciones[indice]}")
    plt.axis('off')
    plt.show()

# Mostrar la primera imagen del conjunto de prueba con su predicción
mostrar_prediccion(X_test, y_test, clases_predichas_ft, indice)


In [ ]:
# 9. Evaluación Más Detallada (Reporte de Clasificación y Matriz de Confusión)
# ------------------------------------

# Imprimir el reporte de clasificación
print("Reporte de Clasificación después del Fine-Tuning:\n")
print(classification_report(y_test, clases_predichas_ft))

# Generar la matriz de confusión
cm_ft = confusion_matrix(y_test, clases_predichas_ft)

# Visualizar la matriz de confusión
plt.figure(figsize=(10,8))
sns.heatmap(cm_ft, annot=True, fmt='d', cmap='Greens', 
            xticklabels=np.unique(y_train), 
            yticklabels=np.unique(y_train))
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión después del Fine-Tuning')
plt.show()